<a href="https://colab.research.google.com/github/vinicius-vargas/robust-market-screener/blob/main/momentum_checking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
pip install -q yfinance setuptools pandas-datareader yahooquery plotly

In [103]:
# @title
### Setting up libraries
import requests
from io import BytesIO
import numpy as np
import pandas as pd
import yfinance as yf
import yahooquery as yq
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
from google.colab import  drive
import warnings
import plotly.express as px

warnings.filterwarnings("ignore")

# Adjust Cientific Notation - Importante to get correct coefficients
# pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [104]:
# @title
# Date Range
start_train = '2024-01-01'
end_test = (datetime.today() - timedelta(days=3)).strftime('%Y-%m-%d')

months = 9


# Tickers of Global Indexes
assets = [
    'EMB'        # Renda Fixa Emergentes
    ,'GC=F'       # Ouro
    ,'BCD'       # GSCI ETF commodities - MATB11
#    ,'BTC-USD'   # Bitcoin Cryptocurrency
    ,'^GSPC'      # ^GSPC - S&P 500 - IVVB11
    ,'IMTM'       # MSCI s/ US
    ,'BRL=X'      # USD vs BRL
    ,'^BVSP'      # Ibovespa
]
assets.sort()

In [105]:
# @title
# Lista de URLs dos arquivos Excel - IMAB - Desde 2003
urls = [
    'https://adata-precos-prod.s3.amazonaws.com/arquivos/indices-historico/IMAB5-HISTORICO.xls',    # IMAB-5 - Comprar IMA-B5-P2 (B5P211)
    'https://adata-precos-prod.s3.amazonaws.com/arquivos/indices-historico/IRFM-HISTORICO.xls',     # IRF-M  - Comprar IRF-M-P2 (IRFM11)
    'https://adata-precos-prod.s3.amazonaws.com/arquivos/indices-historico/IMAS-HISTORICO.xls'      # IMA-S  - Selic
    ]

# Nome das séries correspondentes às URLs
series_names = [
    'IMA_B5P2',
    'IRF_MP2',
    'IMA_S'
]

# DataFrame final
df_fixa = pd.DataFrame()

try:
    for url, series_name in zip(urls, series_names):
        # Fazendo o download do arquivo
        response = requests.get(url)
        response.raise_for_status()  # Verifica se houve erros no download

        # Lendo o conteúdo baixado diretamente em um DataFrame
        data = BytesIO(response.content)
        df = pd.read_excel(data)

        # Selecionando apenas as colunas de interesse
        df = df[['Data de Referência', 'Número Índice']]

        # Renomeando a coluna de valores para o nome da série
        df = df.rename(columns={'Número Índice': series_name})

        # Mesclando com o DataFrame final usando a coluna 'Data de Referência' como chave
        if df_fixa.empty:
            df_fixa = df
        else:
            df_fixa = pd.merge(df_fixa, df, on='Data de Referência', how='inner')


    # Renomeando a coluna 'Data de Referência' para 'date' e configurando como índice
    df_fixa = (
        df_fixa
        .rename(columns={'Data de Referência': 'date'})
        .assign(
            date=lambda x: pd.to_datetime(x['date']).dt.tz_localize('UTC')
        )
        [(lambda x: x['date'] >= start_train)]
        .set_index('date')
    )

except requests.exceptions.RequestException as e:
    print(f'Erro ao baixar um dos arquivos: {e}')
except Exception as e:
    print(f'Erro ao processar os arquivos: {e}')

In [106]:
# @title
### Downloading assets data
data = (
    yf.download(assets, start = start_train, end = end_test)
    .loc[:,('Close', slice(None))]
    .droplevel(level=0, axis=1)
    [lambda x: x.index.dayofweek < 5]
    .assign(
        EMB = lambda x: x['EMB'] * x['BRL=X']
        ,GLD = lambda x: x['GC=F'] * x['BRL=X']
        ,CMDT = lambda x: x['BCD'] * x['BRL=X']
        ,SP500 = lambda x: x['^GSPC'] * x['BRL=X']
        ,IMTM = lambda x: x['IMTM'] * x['BRL=X']
    )
    .drop(
        [
            'GC=F'
            ,'BCD'
            ,'^GSPC'
            ,'BRL=X'
        ]
        , axis=1
    )
    .fillna(method='ffill')
)
data.index = data.index.tz_localize('UTC')

data = pd.concat([data, df_fixa], axis = 1).fillna(method='ffill')

# Concat with dividends dataframe
raw_data = data.pct_change().dropna()
log_data =  np.log1p(data.pct_change()).dropna()

[*********************100%***********************]  7 of 7 completed


In [107]:
# @title
# Supondo que 'raw_data' já esteja calculado conforme o código fornecido
# Filtrar os dados para os últimos 12 meses
data_12m = (
    raw_data
    .loc[raw_data.index >= (raw_data.index.max() - pd.DateOffset(months=months))]
    [['IMA_B5P2',	'IRF_MP2',	'IMA_S', 'EMB']]
)

# Calcular o retorno acumulado:
# (1 + retorno diário).produto acumulado - 1
cumulative_returns = (1 + data_12m).cumprod() - 1

# Converter o DataFrame para o formato longo (long format)
df_plot = cumulative_returns.reset_index().melt(id_vars='index',
                                                var_name='Ativo',
                                                value_name='Retorno Acumulado')
df_plot.rename(columns={'index': 'Data'}, inplace=True)

# Criar o gráfico de linhas usando Plotly Express
fig = px.line(
    df_plot
    ,x='Data'
    ,y='Retorno Acumulado'
    ,color='Ativo'
    ,title= f'Renda Fixa - Retorno Acumulado (M-{months})'
).update_layout(template='plotly_white')

fig.show()

In [108]:
# @title
# Supondo que 'raw_data' já esteja calculado conforme o código fornecido
# Filtrar os dados para os últimos 12 meses
data_12m = (
    raw_data
    .loc[raw_data.index >= (raw_data.index.max() - pd.DateOffset(months=months))]
    [['IMTM','^BVSP', 'SP500', 'EMB']]
)

# Calcular o retorno acumulado:
# (1 + retorno diário).produto acumulado - 1
cumulative_returns = (1 + data_12m).cumprod() - 1

# Converter o DataFrame para o formato longo (long format)
df_plot = cumulative_returns.reset_index().melt(id_vars='index',
                                                var_name='Ativo',
                                                value_name='Retorno Acumulado')
df_plot.rename(columns={'index': 'Data'}, inplace=True)

# Criar o gráfico de linhas usando Plotly Express
fig = px.line(
    df_plot
    ,x='Data'
    ,y='Retorno Acumulado'
    ,color='Ativo'
    ,title= f'Renda Variável - Retorno Acumulado (M-{months})'
).update_layout(template='plotly_white')

fig.show()

In [109]:
# @title
# Supondo que 'raw_data' já esteja calculado conforme o código fornecido
# Filtrar os dados para os últimos 12 meses
data_12m = (
    raw_data
    .loc[raw_data.index >= (raw_data.index.max() - pd.DateOffset(months=months))]
    [['GLD',	'CMDT']]
)

# Calcular o retorno acumulado:
# (1 + retorno diário).produto acumulado - 1
cumulative_returns = (1 + data_12m).cumprod() - 1

# Converter o DataFrame para o formato longo (long format)
df_plot = cumulative_returns.reset_index().melt(id_vars='index',
                                                var_name='Ativo',
                                                value_name='Retorno Acumulado')
df_plot.rename(columns={'index': 'Data'}, inplace=True)

# Criar o gráfico de linhas usando Plotly Express
fig = px.line(
    df_plot
    ,x='Data'
    ,y='Retorno Acumulado'
    ,color='Ativo'
    ,title= f'Commodities - Retorno Acumulado (M-{months})'
).update_layout(template='plotly_white')

fig.show()

In [110]:
# @title
# Supondo que 'raw_data' já esteja calculado conforme o código fornecido
# Filtrar os dados para os últimos 12 meses
data_12m = (
    raw_data
    .loc[raw_data.index >= (raw_data.index.max() - pd.DateOffset(months=months))]
    [['IMTM','^BVSP', 'SP500', 'EMB', 'GLD',	'CMDT']]
)

# Calcular o retorno acumulado:
# (1 + retorno diário).produto acumulado - 1
cumulative_returns = (1 + data_12m).cumprod() - 1

# Converter o DataFrame para o formato longo (long format)
df_plot = cumulative_returns.reset_index().melt(id_vars='index',
                                                var_name='Ativo',
                                                value_name='Retorno Acumulado')
df_plot.rename(columns={'index': 'Data'}, inplace=True)

# Criar o gráfico de linhas usando Plotly Express
fig = px.line(
    df_plot
    ,x='Data'
    ,y='Retorno Acumulado'
    ,color='Ativo'
    ,title= f'Commodities - Retorno Acumulado (M-{months})'
).update_layout(template='plotly_white')

fig.show()